In [26]:
%load_ext IPython.extensions.autoreload
%autoreload 2

The IPython.extensions.autoreload extension is already loaded. To reload it, use:
  %reload_ext IPython.extensions.autoreload


In [30]:
import sys

sys.path.append('../model')

from data import FinData
from preprocessing import train_valid_split
from model import CatboostFinModel

In [31]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import datetime as dt

# from model.data import FinData
# from model.preprocessing import train_valid_split
# from ..model.model import CatboostFinModel

In [32]:
data = FinData("../datasets/T_yandex_10min.csv")

data.insert_shifts_norms()
data.insert_rolling_means()
data.insert_exp_rolling_means()
data.insert_stochastic_oscillator()
data.insert_high_low_diff()
data.insert_holidays()
data.insert_rsi()
data.insert_bollinger()
data.insert_random_prediction()

data.make_binary_class_target()

In [33]:
numeric = data.numeric_features

cat = data.cat_features

args = {"iterations" : 10000, 
        "depth" : 5, 
        "learning_rate" : 0.01, # тут слегка неадекватные параметры, которые можно менять 
        "use_best_model" : True, 
        "l2_leaf_reg" : 200,
        "loss_function" : 'CrossEntropy', 
        "eval_metric" : 'Accuracy', 
        "cat_features" : cat, 
        "random_state" : 42,
        "early_stopping_rounds" : 5000}


model = CatboostFinModel(args)
model.cross_validation(data.df, cat, n_samples=5)

fist_date: 2024-08-12 04:00:00 - last_date: 2024-10-12 04:00:00
const_acc: 0.5226130653266332
fist_date: 2024-08-27 04:00:00 - last_date: 2024-10-27 04:00:00
const_acc: 0.5621890547263682
fist_date: 2024-08-28 04:00:00 - last_date: 2024-10-28 04:00:00
const_acc: 0.6206896551724138
fist_date: 2024-09-30 04:00:00 - last_date: 2024-11-30 04:00:00
const_acc: 0.5125628140703518
(199,)
On trial 4 with date 2024-09-30 00:00:00 got accuracy 0.5577889447236181
fist_date: 2024-08-03 07:00:00 - last_date: 2024-10-03 07:00:00
const_acc: 0.5829145728643216
fist_date: 2024-09-13 04:00:00 - last_date: 2024-11-13 04:00:00
const_acc: 0.6030150753768844
fist_date: 2024-08-02 07:00:00 - last_date: 2024-10-02 07:00:00
const_acc: 0.5829145728643216
fist_date: 2024-09-22 04:00:00 - last_date: 2024-11-22 04:00:00
const_acc: 0.5326633165829145
(199,)
On trial 3 with date 2024-09-22 00:00:00 got accuracy 0.6130653266331658
fist_date: 2024-09-05 04:00:00 - last_date: 2024-11-05 04:00:00
const_acc: 0.60891089108

0.5914406864162293